# Análises das bases do BGG 

In [140]:
import pandas as pd
import plotly.express as px


In [239]:
games = pd.read_csv('archive/games.csv')
mechanics = pd.read_csv('archive/mechanics.csv')
subcat = pd.read_csv('archive/subcategories.csv')



In [213]:
cat = ['YearPublished', 'Cat:Thematic', 'Cat:Strategy', 'Cat:War', 
                 'Cat:Family', 'Cat:CGS', 'Cat:Abstract', 'Cat:Party', 'Cat:Childrens']

games_cat = games[cat].groupby('YearPublished').sum().reset_index()
games_cat = games_cat[games_cat['YearPublished']>=1900]

In [143]:
# Transformando o DataFrame em formato longo para melhor visualização com plotly
df_long_cat = games_cat.melt(id_vars='YearPublished', 
                           value_vars=['Cat:Thematic', 'Cat:Strategy', 'Cat:War', 
                                       'Cat:Family', 'Cat:CGS', 'Cat:Abstract', 
                                       'Cat:Party', 'Cat:Childrens'], 
                           var_name='Category', 
                           value_name='Count')

# Criando o gráfico de linha com Plotly
fig = px.line(df_long_cat, x='YearPublished', y='Count', color='Category', 
              title='Linha do Tempo das Categorias de Jogos',
              labels={'YearPublished': 'Ano de Publicação', 'Count': 'Quantidade de Jogos'})

# Exibindo o gráfico
fig.show()


C:\Users\isabelly.almeida\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [ ]:
len(mechanics.columns.to_list()[1:])
#games.merge(mechanics, how='left')
games[(games['Cat:Strategy'] >= 1) & (games['YearPublished'] >= 2007)].sort_values(by='Rank:strategygames', ascending=False)


In [255]:
games_subcat = games.merge(subcat, how='inner')

subcats = ['YearPublished', 'Exploration', 'Miniatures', 'Territory Building', 'Card Game',
       'Educational', 'Puzzle', 'Collectible Components', 'Word Game',
       'Print & Play', 'Electronic']

games_subcat = games_subcat[subcats].groupby('YearPublished').sum().reset_index()
games_subcat = games_subcat[(games_subcat['YearPublished']>=1900) & (games_subcat['YearPublished']<=2019)]


In [146]:
df_long_sub = games_subcat.melt(id_vars = 'YearPublished',
                            value_vars = ['Exploration', 'Miniatures', 'Territory Building', 'Card Game',
                                        'Educational', 'Puzzle', 'Collectible Components', 'Word Game',
                                        'Print & Play', 'Electronic'],
                            var_name = 'subcats',
                            value_name = 'n')

fig2 = px.line(df_long_sub, x='YearPublished', y='n', color='subcats', 
              title='Linha do Tempo das Subcategorias de Jogos',
              labels={'YearPublished': 'Ano de Publicação', 'Count': 'Quantidade de Jogos'})

fig2.show()

C:\Users\isabelly.almeida\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [175]:
result = pd.DataFrame(columns=['Category'] + subcats[1:])

games_full = games.merge(subcat)

for cat in cat[1:]:
    temp_df = games_full[games_full[cat] == 1][subcats[1:]].sum().reset_index()
    temp_df.columns = ['Subcategory', 'Count']
    temp_df['Category'] = cat
    result = pd.concat([result, temp_df.pivot(index='Category', columns='Subcategory', values='Count').reset_index()])

# Preencher valores NaN com 0
result = result.fillna(0)

result


,Category,Exploration,Miniatures,Territory Building,Card Game,Educational,Puzzle,Collectible Components,Word Game,Print & Play,Electronic


In [158]:
result = result.set_index('Category').unstack().reset_index()

result.rename(columns = {'level_0':'subcat', 'Category':'cat', 0:'n'}, inplace=True)


In [163]:
fig = px.scatter(result, y="subcat", x="cat", size='n',
                 labels={'cat': 'Categorias', 'subcat': 'Subcategorias'})
#fig.update_traces(marker_size='n')
fig.show()

In [ ]:
mechanic = mechanics.columns.tolist()

result = pd.DataFrame(columns=['Category'] + mechanic[1:])

games_full_m = games.merge(mechanics)


for cat in cat[1:]:
    temp_df = games_full_m[games_full_m[cat] == 1][mechanic[1:]].sum().reset_index()
    temp_df.columns = ['mechanic', 'Count']
    temp_df['Category'] = cat
    result = pd.concat([result, temp_df.pivot(index='Category', columns='mechanic', values='Count').reset_index()])



# # Preencher valores NaN com 0
result = result.fillna(0)

result

In [215]:
result = result.set_index('Category').unstack().reset_index()

result.rename(columns = {'level_0':'mechanic', 'Category':'cat', 0:'n'}, inplace=True)


In [236]:
fig = px.scatter(result[result['n']>2], y="mechanic", x="cat", size='n',
                 labels={'cat': 'Categorias', 'mechanic': 'Mecânicas'})
#fig.update_traces(marker_size='n')
# Configurando o layout para definir o tamanho do gráfico
fig.update_layout(width=800, height=2500)
fig.show()

In [263]:
subcats = subcat.columns.to_list()
result = pd.DataFrame(columns=['subcategory'] + mechanic[1:])

subcat_mech = subcat.merge(mechanics)

for subcats in subcats[1:]:
    temp_df = subcat_mech[subcat_mech[subcats] == 1][mechanic[1:]].sum().reset_index()
    temp_df.columns = ['mechanic', 'Count']
    temp_df['subcategory'] = subcats
    result = pd.concat([result, temp_df.pivot(index='subcategory', columns='mechanic', values='Count').reset_index()])



# Preencher valores NaN com 0
result = result.fillna(0)
result
result = result.set_index('subcategory').unstack().reset_index()

result.rename(columns = {'level_0':'mechanic', 'subcategory':'subcat', 0:'n'}, inplace=True)


fig = px.scatter(result[result['n']<2000], y="mechanic", x="subcat", size='n',
                 labels={'subcat': 'subcategory', 'mechanic': 'Mecânicas'})
#fig.update_traces(marker_size='n')
# Configurando o layout para definir o tamanho do gráfico
fig.update_layout(width=800, height=2500)
fig.show()


C:\Users\isabelly.almeida\AppData\Local\Temp\ipykernel_24972\4012896178.py:15: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,subcategory,Alliances,Area Majority / Influence,Auction/Bidding,Dice Rolling,Hand Management,Simultaneous Action Selection,Trick-taking,Hexagon Grid,Once-Per-Game Abilities,...,Contracts,Passed Action Token,King of the Hill,Action Retrieval,Force Commitment,Rondel,Automatic Resource Growth,Legacy Game,Dexterity,Physical
0,Exploration,0,91,28,373,196,37,0,57,7,...,8,0,3,8,2,5,0,7,13,15
0,Miniatures,4,90,20,788,183,75,0,80,8,...,2,1,5,6,3,1,1,4,19,2
0,Territory Building,4,268,51,136,126,26,0,31,1,...,8,0,4,2,4,5,0,0,3,3
0,Card Game,8,287,330,686,2912,468,383,12,8,...,29,1,4,16,2,4,0,8,166,166
0,Educational,1,27,14,90,105,14,7,8,0,...,3,0,0,1,0,0,0,0,16,9
0,Puzzle,0,22,7,93,68,17,1,11,1,...,8,0,0,1,0,2,3,0,50,13
0,Collectible Components,1,17,5,120,130,14,1,5,0,...,0,0,0,0,0,0,0,0,20,9
0,Word Game,0,4,5,65,53,32,2,1,0,...,0,0,0,0,0,0,0,0,20,35
0,Print & Play,1,51,20,228,134,36,9,47,1,...,2,0,0,2,0,2,0,4,3,17
0,Electronic,0,2,12,21,9,7,0,0,0,...,0,0,0,0,0,0,0,0,45,8
